In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType
from pyspark.sql.functions import round, col

import pyspark.sql.functions as F

spark = SparkSession.builder.appName('ProjectInsurance').getOrCreate()

spark.conf.set("fs.azure.account.key.pprojecthealthcare456.dfs.core.windows.net", "EKJeYMicvDEsPEIdi+gJzlc0KxwsUivTTIVR2jHeEBM79z1Cd7cOEP0Z5UVv0/QlcYwrBHsrdBm3+AStDru99w==")

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("sex", StringType(), True),
    StructField("bmi", FloatType(), True),
    StructField("children", IntegerType(), True),
    StructField("smoker", StringType(), True),
    StructField("region", StringType(), True),
    StructField("charges", FloatType(), True)
])

Healthcare_schema = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .csv("abfss://raw@pprojecthealthcare456.dfs.core.windows.net/insurance.csv")

Insurance = Healthcare_schema.withColumnRenamed("age", "Age") \
    .withColumnRenamed("sex", "Sex") \
    .withColumnRenamed("bmi", "BMI") \
    .withColumnRenamed("children", "Children") \
    .withColumnRenamed("smoker", "Smoker") \
    .withColumnRenamed("region", "Region") \
    .withColumnRenamed("charges", "Charges")

# Filter to get age greater than 18
filtered_df = Insurance.filter(Insurance['Age'] > 18)

df_new = Insurance.withColumn('rounded_charges', round(Insurance.Charges, 0))

# Filter to get Charges greater than 10000
df_new = df_new.filter(col('rounded_charges') >= 10000)


# Count rows where 'Smoker' = no and 'Smoker' > yes
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
result = df_new.groupBy('Region').agg(
    cnt_cond(F.col('Smoker') == 'no').alias('non-smoker_cnt'),
    cnt_cond(F.col('Smoker') == 'yes').alias('smoker_cnt')
)


df_new.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@pprojecthealthcare456.dfs.core.windows.net/insurance_transformed.csv")
result.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://processed@pprojecthealthcare456.dfs.core.windows.net/insurance_Filter.csv")

